In [36]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

"""Accuracy measures Logistic, not R2 as in the other models."""
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
"""__________________"""

from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
import datetime
from sklearn.preprocessing import StandardScaler
from scipy.stats import boxcox
import numpy as np

pd.set_option('display.max_columns', None)

import os

from statsmodels.formula.api import ols
import statsmodels.api as sm

import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm


from imblearn.over_sampling import SMOTE


from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import TomekLinks




## Instructions

### Apply SMOTE for upsampling the data

* Use logistic regression to fit the model and compute the accuracy of the model.
* Use decision tree classifier to fit the model and compute the accuracy of the model.
* Compare the accuracies of the two models.

### Apply TomekLinks for downsampling
* It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.
* Use logistic regression to fit the model and compute the accuracy of the model.
* Use decision tree classifier to fit the model and compute the accuracy of the model.

Compare the accuracies of the two models.
You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.

## STEP 1: Clean and get the data ready for model

In [2]:
churnData = pd.read_csv("Customer-Churn.csv")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
churnData.isna().sum()
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
numerical=churnData.select_dtypes(np.number)
categorical=churnData.select_dtypes(object)


In [5]:
for i in categorical.columns:
    print (categorical[i].value_counts())

Male      3555
Female    3488
Name: gender, dtype: int64
No     3641
Yes    3402
Name: Partner, dtype: int64
No     4933
Yes    2110
Name: Dependents, dtype: int64
Yes    6361
No      682
Name: PhoneService, dtype: int64
No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64
No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64
No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64
No                     2785
Yes                    2732
No internet service    1526
Name: StreamingMovies, dtype: int64
Month-to-month    3875
Two year          1695
One year          1473
N

In [6]:
categorical["TotalCharges"] = pd.to_numeric(categorical["TotalCharges"], errors="coerce")

In [7]:
numerical = pd.concat([numerical, pd.to_numeric(categorical["TotalCharges"], errors="coerce")], axis=1)
categorical=categorical.drop("TotalCharges",axis=1)

In [8]:
for i in categorical.columns:
    categorical[i] = LabelEncoder().fit_transform(categorical[i])

In [11]:
model = pd.concat([numerical, categorical], axis=1)

In [17]:
model.isna().sum()

SeniorCitizen        0
tenure               0
MonthlyCharges       0
TotalCharges        11
gender               0
Partner              0
Dependents           0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
Churn                0
dtype: int64

In [19]:
model["TotalCharges"]=model["TotalCharges"].fillna(np.mean(model["TotalCharges"]))

## STEP 2: Apply Smote

In [20]:
model["Churn"].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [21]:
smote = SMOTE()
X = model.drop("Churn",axis=1)
y=model["Churn"]

X_sm,y_sm=smote.fit_resample(X,y)

In [26]:
model_smote = pd.concat([pd.DataFrame(X_sm), pd.DataFrame(y_sm)], axis=1)

In [28]:
model_smote["Churn"].value_counts()

0    5174
1    5174
Name: Churn, dtype: int64

In [30]:
#Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(model_smote.drop("Churn", axis=1), model_smote["Churn"], test_size=0.3)
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
lr_acc

0.7993558776167472

In [33]:
#Decision Tree
dtr = DecisionTreeClassifier()
dtr.fit(X_train, y_train)
dtr_pred = dtr.predict(X_test)
dtr_acc = accuracy_score(y_test, dtr_pred)
dtr_acc

0.7874396135265701

## STEP 3: Apply TomekLinks 

In [35]:
model["Churn"].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [37]:
tomek = TomekLinks()
X_resampled, y_resampled = tomek.fit_resample(model.drop("Churn", axis=1), model["Churn"])
model_tomek = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
model_tomek.columns = model.columns

In [39]:
model_tomek["Churn"].value_counts()

0    4605
1    1869
Name: Churn, dtype: int64

In [40]:
#Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(model_tomek.drop("Churn", axis=1), model_tomek["Churn"], test_size=0.3)
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
lr_acc

0.8208955223880597

In [41]:
#Decision Tree
dtr = DecisionTreeClassifier()
dtr.fit(X_train, y_train)
dtr_pred = dtr.predict(X_test)
dtr_acc = accuracy_score(y_test, dtr_pred)
dtr_acc

0.7534740092640247

In [42]:
X_resampled, y_resampled = tomek.fit_resample(model_tomek.drop("Churn", axis=1), model_tomek["Churn"])
model_tomek_2 = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
model_tomek_2.columns = model_tomek.columns

In [43]:
model_tomek_2["Churn"].value_counts()

0    4436
1    1869
Name: Churn, dtype: int64

In [44]:
#Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(model_tomek_2.drop("Churn", axis=1), model_tomek_2["Churn"], test_size=0.3)
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
lr_acc

0.8060253699788583

In [45]:
#Decision Tree
dtr = DecisionTreeClassifier()
dtr.fit(X_train, y_train)
dtr_pred = dtr.predict(X_test)
dtr_acc = accuracy_score(y_test, dtr_pred)
dtr_acc

0.7447145877378436